In [1]:
import numpy as np
import pandas as pd

In [2]:
# initialization
A = np.array([[0.47468, 0.52532], [0.51656, 0.48344]])
# B = np.array([[0.1, 0.4, 0.5], [0.6, 0.3, 0.1]])
PI = np.array([0.51316, 0.48684])
T = 1536 # length of observation sequence
N = 2 # number of states in model
M = 30 # number of observations model
# V = {1, 2, 3} # set of possible observations
# O = np.array([2, 2, 3, 1, 3, 2])

In [3]:
def calcAlphaBeta(A, B, O, PI, T, N):
    # initialization ALPHA
    c = np.zeros((T))
    alpha = np.zeros((T, N))
    for i in range(N):
        alpha[0,i] = PI[i]*B[i,O[0]-1]
        c[0] = c[0] + alpha[0,i]
        
    # scale alpha1
    c[0] = 1/c[0]
    for i in range(N):
        alpha[0,i] = c[0]*alpha[0,i]
    
    # induction
    for t in range(T-1):
        c[t+1] = 0
        for j in range(N):
            alpha[t+1,j] = 0
            for i in range(N):
                alpha[t+1,j] = alpha[t+1,j] + alpha[t,i] * A[i,j]
            alpha[t+1,j] = alpha[t+1,j] * B[j,O[t+1]-1]
            c[t+1] = c[t+1] + alpha[t+1,j]
        
        # scale alpha
        c[t+1] = 1/c[t+1]
        for i in range(N):
            alpha[t+1,i] = c[t+1] * alpha[t+1,i]
            
    # initialization BETA
    beta = np.zeros((T, N))
    for i in range(N):
        beta[T,i] = c[T]
        
    # induction
    for t in range(T-2, -1, -1):
        for i in range(N):
            beta[t,i] = 0
            for j in range(N):
                beta[t,i] = beta[t,i] + A[i,j] * B[j,O[t+1]-1] * beta[t+1,j]
            
            # scale beta
            beta[t,i] = c[t] * beta[t,i]
    
    return alpha, beta, c

In [4]:
def calcLogP(T, c):
    logp = 0
    for i in range(T):
        logp = logp + np.log(c[i])
    
    return -logp

In [5]:
def calcGammaDigamma(alpha, beta, T, N, A, B, O):
    gamma = np.zeros((T, N))
    di_gamma = np.zeros((T-1, N, N))
    for t in range(T):
        denom = 0
        for i in range(N):
            for j in range(N):
                denom = denom + alpha[t,i] * A[i,j] * B[j,O[t+1]-1] * beta[t+1,j]
        
        for i in range(N):
            gamma[t,i] = 0
            for j in range(N):
                di_gamma[t,i,j] = (alpha[t,i] * A[i,j] * B[j,O[t+1]-1] * beta[t+1,j])/denom
                gamma[t,i] = gamma[t,i] + di_gamma[t,i,j]
    
    return gamma, di_gamma

In [6]:
def reEstimate(gamma, di_gamma, T, N, M, O):
    # re-estimate PI
    PI1 = np.zeros((N))
    for i in range(N):
        PI1[i] = gamma[0,i]

    # re-estimate A
    a = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            num = 0
            den = 0
            for t in range(T-1):
                num = num + di_gamma[t,i,j]
                den = den + gamma[t,i]
            a[i,j] = num/den
        
    # re-estimate B
    b = np.zeros((N, M))
    for j in range(N):
        for k in range(M):
            num = 0
            den = 0
            for t in range(T-1):
                if O[t]-1 == k:
                    num = num + gamma[t,j]
                den = den + gamma[t,j]
            b[j,k] = num/den
            
    return a, b, PI1

In [7]:
def prob(A, B, PI, O, T, N, M):
    maxIter = 200
    iteration = 0
    old_logp = 0
    logp = np.log(0)
    while 1:
        iter = iter + 1
        alpha, beta, c = calcAlphaBeta(A, B, O, PI, T, N)
        gamma, di_gamma = calcGammaDigamma(alpha, beta, T, N, A, B, O)
        old_logp = logp
        logp = calcLogP(T, c)
        a, b, PI1 = reEstimate(gamma, di_gamma, T, N, M, O)
        A, B, PI = a, b, PI1
        if logp <= old_logp or iteration == maxIter:
            break
    return A, B, PI, logp